In [1]:
import pandas as pd
import numpy as np
import itertools
import datetime
import pandas_gbq
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
# %load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
project_id = "perceptive-ivy-290216"

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
query2=f"""
SELECT
*
FROM `perceptive-ivy-290216.f1_api.race_lap_time`  A
# WHERE A.Year=2024
# AND A.GP="Monaco Grand Prix"
# AND A.DRIVER='HAM'
ORDER BY LapNumber, LapStartTime
"""
track3=pandas_gbq.read_gbq(query2,project_id,dialect='standard')

Downloading: 100%|██████████|


In [7]:
track2=track3[(track3["GP"]=='Belgian Grand Prix')&(track3["Year"]==2024)]
track2.head()
year=track2['Year'].iloc[0]
gp=track2['GP'].iloc[0]

In [8]:
track2['LapTime']= pd.to_timedelta(track2["LapTime"])

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_4782/4173085553.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  track2['LapTime']= pd.to_timedelta(track2["LapTime"])


In [9]:
# track2['LapTime'].dt.total_seconds().min()*1.07
track2=track2[track2['LapNumber']!=1.0]

In [10]:
quicklaps=track2[track2["LapTime"]<track2['LapTime'].min()*1.07]
quicklaps.head()

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP
20477,0 days 00:58:57.253000,LEC,16,0 days 00:01:50.240000,2.0,1.0,NaT,NaT,0 days 00:00:31.831000,0 days 00:00:48.675000,0 days 00:00:29.734000,0 days 00:57:38.844000,0 days 00:58:27.519000,0 days 00:58:57.253000,303.0,203.0,215.0,NaN,True,MEDIUM,2.0,True,Ferrari,0 days 00:57:07.013000,NaT,1,1.0,False,,False,True,2024,Belgian Grand Prix
20480,0 days 00:58:57.650000,HAM,44,0 days 00:01:49.849000,2.0,1.0,NaT,NaT,0 days 00:00:31.284000,0 days 00:00:49.211000,0 days 00:00:29.354000,0 days 00:57:39.019000,0 days 00:58:28.230000,0 days 00:58:57.584000,329.0,202.0,220.0,307.0,True,MEDIUM,2.0,True,Mercedes,0 days 00:57:07.801000,NaT,1,2.0,False,,False,True,2024,Belgian Grand Prix
20485,0 days 00:58:58.780000,PER,11,0 days 00:01:50.240000,2.0,1.0,NaT,NaT,0 days 00:00:31.365000,0 days 00:00:49.207000,0 days 00:00:29.668000,0 days 00:57:39.872000,0 days 00:58:29.079000,0 days 00:58:58.747000,325.0,202.0,217.0,308.0,True,MEDIUM,2.0,True,Red Bull Racing,0 days 00:57:08.540000,NaT,1,3.0,False,,False,True,2024,Belgian Grand Prix
20491,0 days 00:58:59.513000,PIA,81,0 days 00:01:50.164000,2.0,1.0,NaT,NaT,0 days 00:00:31.056000,0 days 00:00:49.480000,0 days 00:00:29.628000,0 days 00:57:40.397000,0 days 00:58:29.877000,0 days 00:58:59.505000,335.0,201.0,216.0,313.0,True,MEDIUM,2.0,True,McLaren,0 days 00:57:09.349000,NaT,1,4.0,False,,False,True,2024,Belgian Grand Prix
20494,0 days 00:59:00.024000,RUS,63,0 days 00:01:50.160000,2.0,1.0,NaT,NaT,0 days 00:00:31.075000,0 days 00:00:49.700000,0 days 00:00:29.385000,0 days 00:57:40.899000,0 days 00:58:30.599000,0 days 00:58:59.984000,328.0,199.0,220.0,312.0,True,MEDIUM,2.0,True,Mercedes,0 days 00:57:09.864000,NaT,1,5.0,False,,False,True,2024,Belgian Grand Prix


In [11]:
transformed_laps = quicklaps.copy()
transformed_laps.loc[:, "LapTime (s)"] = quicklaps["LapTime"].dt.total_seconds()

# order the team from the fastest (lowest median lap time) tp slower
team_order = (
    transformed_laps[["Driver", "LapTime (s)"]].groupby("Driver").median()["LapTime (s)"].sort_values().index
)
print(team_order)

Index(['HAM', 'RUS', 'LEC', 'NOR', 'PIA', 'VER', 'PER', 'SAI', 'RIC', 'SAR',
       'ALB', 'ALO', 'TSU', 'OCO', 'MAG', 'STR', 'GAS', 'BOT', 'HUL'],
      dtype='object', name='Driver')


In [12]:
transformed_laps['median'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Driver']).transform('median')
transformed_laps.tail()

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP,LapTime (s),median
437887,0 days 02:16:18.772000,MAG,20,0 days 00:01:47.848000,44.0,2.0,NaT,NaT,0 days 00:00:30.979000,0 days 00:00:48.058000,0 days 00:00:28.811000,0 days 02:15:01.882000,0 days 02:15:49.940000,0 days 02:16:18.751000,317.0,204.0,221.0,317.0,True,HARD,27.0,True,Haas F1 Team,0 days 02:14:30.924000,NaT,1,15.0,False,,False,True,2024,Belgian Grand Prix,107.848,109.5115
437891,0 days 02:16:22.779000,BOT,77,0 days 00:01:47.516000,44.0,3.0,NaT,NaT,0 days 00:00:31.022000,0 days 00:00:47.699000,0 days 00:00:28.795000,0 days 02:15:06.241000,0 days 02:15:53.940000,0 days 02:16:22.735000,316.0,208.0,222.0,313.0,False,MEDIUM,9.0,True,Kick Sauber,0 days 02:14:35.263000,NaT,1,16.0,False,,False,True,2024,Belgian Grand Prix,107.516,109.5910
437893,0 days 02:16:28.878000,TSU,22,0 days 00:01:47.969000,44.0,2.0,NaT,NaT,0 days 00:00:30.937000,0 days 00:00:48.057000,0 days 00:00:28.975000,0 days 02:15:11.802000,0 days 02:15:59.859000,0 days 02:16:28.834000,NaN,208.0,221.0,313.0,True,HARD,30.0,False,RB,0 days 02:14:40.909000,NaT,1,17.0,False,,False,True,2024,Belgian Grand Prix,107.969,109.4150
437899,0 days 02:16:38.198000,SAR,2,0 days 00:01:48.121000,44.0,3.0,NaT,NaT,0 days 00:00:31.021000,0 days 00:00:48.123000,0 days 00:00:28.977000,0 days 02:15:21.087000,0 days 02:16:09.210000,0 days 02:16:38.187000,323.0,202.0,220.0,313.0,False,HARD,20.0,True,Williams,0 days 02:14:50.077000,NaT,1,18.0,False,,False,True,2024,Belgian Grand Prix,108.121,109.1340
437902,0 days 02:16:40.974000,HUL,27,0 days 00:01:48.954000,44.0,3.0,NaT,NaT,0 days 00:00:31.003000,0 days 00:00:48.902000,0 days 00:00:29.049000,0 days 02:15:22.991000,0 days 02:16:11.893000,0 days 02:16:40.942000,314.0,199.0,219.0,NaN,True,MEDIUM,24.0,True,Haas F1 Team,0 days 02:14:52.020000,NaT,1,19.0,False,,False,True,2024,Belgian Grand Prix,108.954,109.6340


In [13]:
fig_pace=px.box(
    transformed_laps.sort_values(by=["median","LapNumber"]),
    x="Driver",
    y="LapTime (s)",
    color='Driver',
    template="xgridoff",
    title="<b>Driver Pace Plot for the {} {}</b>".format(year,gp),
    # palette=team_palette,
    # whiskerprops=dict(color="white"),
    # boxprops=dict(edgecolor="white"),
    # medianprops=dict(color="grey"),
    # capprops=dict(color="white"),
    height=700, width=1200,

color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
          }
)
fig_pace.update_traces(opacity=1)
fig_pace.update_layout(
    title_x=0.5,
    hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow",
    margin=dict(l=50, r=5, t=35, b=50)
)
fig_pace.show()

In [11]:
fig_pace.write_html("{} {} Driver Pace Plot.html".format(year,gp))

In [14]:
#Get MINIMUM OF THE MEDIAN LAP TIMES

fastest_lap = transformed_laps["median"].min()
transformed_laps['Delta']= transformed_laps["median"] - (fastest_lap)
transformed_laps['Delta_Percent']= (transformed_laps["median"]/(fastest_lap)-1)*100
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent']+100
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent'].round(3)
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent'].astype(str)
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent']+"%"
transformed_laps_relative_median=transformed_laps[["Driver","Delta","Delta_Percent"]].drop_duplicates()
transformed_laps_relative_median=transformed_laps_relative_median.sort_values("Delta")
transformed_laps_relative_median

,Driver,Delta,Delta_Percent
20480,HAM,0.0000,100.0%
20494,RUS,0.0350,100.032%
20477,LEC,0.0640,100.059%
20500,NOR,0.1580,100.146%
20491,PIA,0.2010,100.186%
20502,VER,0.2345,100.217%
20485,PER,0.3780,100.35%
20497,SAI,0.3980,100.369%
20507,RIC,0.9030,100.836%
31315,SAR,1.1770,101.09%


In [15]:
for index, row in transformed_laps_relative_median.iterrows():
  if transformed_laps_relative_median.loc[index, 'Delta']==0.0:
    transformed_laps_relative_median.loc[index, 'Delta']=0.001
transformed_laps_relative_median

,Driver,Delta,Delta_Percent
20480,HAM,0.0010,100.0%
20494,RUS,0.0350,100.032%
20477,LEC,0.0640,100.059%
20500,NOR,0.1580,100.146%
20491,PIA,0.2010,100.186%
20502,VER,0.2345,100.217%
20485,PER,0.3780,100.35%
20497,SAI,0.3980,100.369%
20507,RIC,0.9030,100.836%
31315,SAR,1.1770,101.09%


In [16]:
fig_median=px.bar(
    transformed_laps_relative_median,
    x="Driver",
    y="Delta",
    color='Driver',
    template="xgridoff",
    title="<b>Driver Pace (Median Lap Times) for the {} {}</b>".format(year,gp),
    text="Delta_Percent",
    height=700, 
    width=1200,
    color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
          }
)

fig_median.update_traces(textposition='outside')

fig_median.update_traces(opacity=0.85)

fig_median.update_traces( marker_line_color='white',marker_line_width=1.5)

fig_median.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

fig_median.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

fig_median.update_yaxes(range=[2.5, 0])

fig_median.update_layout(
    xaxis={'side': 'top'},
    yaxis={'side': 'left'}
)

fig_median.update_xaxes(title_text='')      # Remove axis titles
fig_median.update_yaxes(title_text='')

fig_median.update_xaxes(
        title_standoff = 75
)

fig_median.update_layout(
    title_x=0.5,
    margin=dict(l=60, r=5, t=110, b=30),
    hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=12,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_median.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)
fig_median.show()

In [15]:
fig_median.write_html("{} {} Driver Median Plot.html".format(year,gp))

In [17]:
#Get MINIMUM OF ALL LAP TIMES
transformed_laps['min_all'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Driver']).transform('min')

fastest_lap_all = transformed_laps["min_all"].min()
transformed_laps['Delta_Min_All']= transformed_laps["min_all"] - (fastest_lap_all)
transformed_laps['Delta_Percent_All']= (transformed_laps["min_all"]/(fastest_lap_all)-1)*100
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All']+100
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All'].round(3)
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All'].astype(str)
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All']+"%"
transformed_laps_relative_min_all=transformed_laps[["Driver","Delta_Min_All","Delta_Percent_All"]].drop_duplicates()
transformed_laps_relative_min_all=transformed_laps_relative_min_all.sort_values("Delta_Min_All")
transformed_laps_relative_min_all

,Driver,Delta_Min_All,Delta_Percent_All
20485,PER,0.000,100.0%
20500,NOR,0.862,100.823%
20491,PIA,1.139,101.088%
20502,VER,1.427,101.363%
20497,SAI,1.663,101.588%
20480,HAM,1.952,101.864%
20504,OCO,2.256,102.155%
20477,LEC,2.312,102.208%
20506,BOT,2.318,102.214%
20494,RUS,2.412,102.304%


In [18]:
for index, row in transformed_laps_relative_min_all.iterrows():
  if transformed_laps_relative_min_all.loc[index, 'Delta_Min_All']==0.0:
    transformed_laps_relative_min_all.loc[index, 'Delta_Min_All']=0.001
transformed_laps_relative_min_all

,Driver,Delta_Min_All,Delta_Percent_All
20485,PER,0.001,100.0%
20500,NOR,0.862,100.823%
20491,PIA,1.139,101.088%
20502,VER,1.427,101.363%
20497,SAI,1.663,101.588%
20480,HAM,1.952,101.864%
20504,OCO,2.256,102.155%
20477,LEC,2.312,102.208%
20506,BOT,2.318,102.214%
20494,RUS,2.412,102.304%


In [19]:
fig_min=px.bar(
    transformed_laps_relative_min_all,
    x="Driver",
    y="Delta_Min_All",
    color='Driver',
    template="xgridoff",
    title="<b>Driver Pace (Fastest Lap Times) for the {} {}</b>".format(year,gp),
    text="Delta_Percent_All",
    height=700, width=1200,
    color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
          }
)

fig_min.update_traces(textposition='outside')

fig_min.update_traces(opacity=0.85)

fig_min.update_traces( marker_line_color='white',marker_line_width=1.5)

fig_min.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

fig_min.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

fig_min.update_yaxes(range=[4.5, 0])

fig_min.update_layout(
    xaxis={'side': 'top'},
    yaxis={'side': 'left'}
)

fig_min.update_xaxes(title_text='')      # Remove axis titles
fig_min.update_yaxes(title_text='')

fig_min.update_xaxes(
        title_standoff = 75
)

fig_min.update_layout(
    title_x=0.5,
    margin=dict(l=60, r=5, t=110, b=30),
    hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=12,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_min.update_yaxes(ticksuffix = "  ")

fig_min.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)

fig_min.show()

In [20]:
fig_min.write_html("{} {} Driver Fastest Lap Plot.html".format(year,gp))

NameError: name 'fig_min' is not defined

In [ ]:
#Multi Race Plot

In [7]:
#2023
# race_multi=["Abu Dhabi Grand Prix","Australian Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Canadian Grand Prix","Dutch Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Japanese Grand Prix","Las Vegas Grand Prix","Mexico City Grand Prix","Miami Grand Prix","Monaco Grand Prix","Pre-Season Testing","Qatar Grand Prix","Saudi Arabian Grand Prix","Singapore Grand Prix","Spanish Grand Prix","São Paulo Grand Prix","United States Grand Prix"]
#2022
# race_multi=["Abu Dhabi Grand Prix","Australian Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Canadian Grand Prix","Dutch Grand Prix","Emilia Romagna Grand Prix","French Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Japanese Grand Prix","Mexico City Grand Prix","Miami Grand Prix","Monaco Grand Prix","Saudi Arabian Grand Prix","Singapore Grand Prix","Spanish Grand Prix","São Paulo Grand Prix","United States Grand Prix"]
#2021
# race_multi=["Abu Dhabi Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Dutch Grand Prix","Emilia Romagna Grand Prix","French Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Mexico City Grand Prix","Monaco Grand Prix","Portuguese Grand Prix","Qatar Grand Prix","Russian Grand Prix","Saudi Arabian Grand Prix","Spanish Grand Prix","Styrian Grand Prix","São Paulo Grand Prix","Turkish Grand Prix","United States Grand Prix"]
#2020
race_multi=["70th Anniversary Grand Prix","Abu Dhabi Grand Prix","Austrian Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Eifel Grand Prix","Emilia Romagna Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Portuguese Grand Prix","Russian Grand Prix","Sakhir Grand Prix","Spanish Grand Prix","Styrian Grand Prix","Turkish Grand Prix","Tuscan Grand Prix"]

year_multi=2020
for i in race_multi:
    track2=track3[(track3["GP"]==i)&(track3["Year"]==year_multi)]
    year=track2['Year'].iloc[0]
    gp=track2['GP'].iloc[0]
    track2['LapTime']= pd.to_timedelta(track2["LapTime"])
    track2=track2[track2['LapNumber']!=1.0]
    quicklaps=track2[track2["LapTime"]<track2['LapTime'].min()*1.07]
    transformed_laps = quicklaps.copy()
    transformed_laps.loc[:, "LapTime (s)"] = quicklaps["LapTime"].dt.total_seconds()

    # order the team from the fastest (lowest median lap time) tp slower
    team_order = (
        transformed_laps[["Driver", "LapTime (s)"]].groupby("Driver").median()["LapTime (s)"].sort_values().index
    )
    print(team_order)
    transformed_laps['median'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Driver']).transform('median')
    transformed_laps.tail()
    fig_pace=px.box(
        transformed_laps.sort_values(by=["median","LapNumber"]),
        x="Driver",
        y="LapTime (s)",
        color='Driver',
        template="xgridoff",
        title="<b>Driver Pace Plot for the {} {}</b>".format(year_multi,i),
        height=700, width=1200,

    color_discrete_map={
                "AIT": "#0082fa",
                 "ALB": "#1E41FF",
                 "BOT": "#00D2BE",
                 "FIT": "#787878",
                 "GAS": "#2B4562",
                 "GIO": "#9B0000",    
                 "GRO": "#787878",               
                 "HAM": "#00D2BE",
                 "HUL": "#F596C8",
                 "KVY": "#2B4562",
                 "LAT": "#0082fa",
                 "LEC": "#DC0000",
                 "MAG": "#787878",
                 "NOR": "#FF8700",
                 "OCO": "#FFF500",
                 "PER": "#F596C8",
                 "RAI": "#9B0000",
                 "RIC": "#FFF500",
                 "RUS": "#0082fa",
                 "SAI": "#FF8700",
                 "STR": "#F596C8",
                 "VER": "#1E41FF",
                 "VET": "#DC0000",
            }
    )
    fig_pace.update_traces(opacity=1)
    fig_pace.update_layout(
        title_x=0.5,
        hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=15)),
        xaxis = dict(tickfont = dict(size=15)),
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow",
        margin=dict(l=50, r=5, t=35, b=50)
    )
    fig_pace.write_html("{} {} Driver Pace Plot.html".format(year_multi,i))
    #Get MINIMUM OF THE MEDIAN LAP TIMES

    fastest_lap = transformed_laps["median"].min()
    transformed_laps['Delta']= transformed_laps["median"] - (fastest_lap)
    transformed_laps['Delta_Percent']= (transformed_laps["median"]/(fastest_lap)-1)*100
    transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent']+100
    transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent'].round(3)
    transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent'].astype(str)
    transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent']+"%"
    transformed_laps_relative_median=transformed_laps[["Driver","Delta","Delta_Percent"]].drop_duplicates()
    transformed_laps_relative_median=transformed_laps_relative_median.sort_values("Delta")
    transformed_laps_relative_median
    for index, row in transformed_laps_relative_median.iterrows():
        if transformed_laps_relative_median.loc[index, 'Delta']==0.0:
         transformed_laps_relative_median.loc[index, 'Delta']=0.001
    transformed_laps_relative_median
    fig_median=px.bar(
        transformed_laps_relative_median,
        x="Driver",
        y="Delta",
        color='Driver',
        template="xgridoff",
        title="<b>Driver Pace (Median Lap Times) for the {} {}</b>".format(year_multi,i),
        text="Delta_Percent",
        height=700, 
        width=1200,
        color_discrete_map={
                "AIT": "#0082fa",
                 "ALB": "#1E41FF",
                 "BOT": "#00D2BE",
                 "FIT": "#787878",
                 "GAS": "#2B4562",
                 "GIO": "#9B0000",    
                 "GRO": "#787878",               
                 "HAM": "#00D2BE",
                 "HUL": "#F596C8",
                 "KVY": "#2B4562",
                 "LAT": "#0082fa",
                 "LEC": "#DC0000",
                 "MAG": "#787878",
                 "NOR": "#FF8700",
                 "OCO": "#FFF500",
                 "PER": "#F596C8",
                 "RAI": "#9B0000",
                 "RIC": "#FFF500",
                 "RUS": "#0082fa",
                 "SAI": "#FF8700",
                 "STR": "#F596C8",
                 "VER": "#1E41FF",
                 "VET": "#DC0000",
            }
    )

    fig_median.update_traces(textposition='outside')

    fig_median.update_traces(opacity=0.85)

    fig_median.update_traces( marker_line_color='white',marker_line_width=1.5)

    fig_median.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

    fig_median.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

    fig_median.update_yaxes(range=[2.5, 0])

    fig_median.update_layout(
        xaxis={'side': 'top'},
        yaxis={'side': 'left'}
    )

    fig_median.update_xaxes(title_text='')      # Remove axis titles
    fig_median.update_yaxes(title_text='')

    fig_median.update_xaxes(
            title_standoff = 75
    )

    fig_median.update_layout(
        title_x=0.5,
        margin=dict(l=60, r=5, t=110, b=30),
        hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=15)),
        xaxis = dict(tickfont = dict(size=15)),
        font=dict(
            family="PT Sans Narrow",
            size=12,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    fig_median.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)

    fig_median.write_html("{} {} Driver Median Plot.html".format(year_multi,i))

    transformed_laps['min_all'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Driver']).transform('min')

    fastest_lap_all = transformed_laps["min_all"].min()
    transformed_laps['Delta_Min_All']= transformed_laps["min_all"] - (fastest_lap_all)
    transformed_laps['Delta_Percent_All']= (transformed_laps["min_all"]/(fastest_lap_all)-1)*100
    transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All']+100
    transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All'].round(3)
    transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All'].astype(str)
    transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All']+"%"
    transformed_laps_relative_min_all=transformed_laps[["Driver","Delta_Min_All","Delta_Percent_All"]].drop_duplicates()
    transformed_laps_relative_min_all=transformed_laps_relative_min_all.sort_values("Delta_Min_All")
    transformed_laps_relative_min_all
    for index, row in transformed_laps_relative_min_all.iterrows():
        if transformed_laps_relative_min_all.loc[index, 'Delta_Min_All']==0.0:
         transformed_laps_relative_min_all.loc[index, 'Delta_Min_All']=0.001
    transformed_laps_relative_min_all
    fig_min=px.bar(
        transformed_laps_relative_min_all,
        x="Driver",
        y="Delta_Min_All",
        color='Driver',
        template="xgridoff",
        title="<b>Driver Pace (Fastest Lap Times) for the {} {}</b>".format(year_multi,i),
        text="Delta_Percent_All",
        height=700, width=1200,
        color_discrete_map={
                "AIT": "#0082fa",
                 "ALB": "#1E41FF",
                 "BOT": "#00D2BE",
                 "FIT": "#787878",
                 "GAS": "#2B4562",
                 "GIO": "#9B0000",    
                 "GRO": "#787878",               
                 "HAM": "#00D2BE",
                 "HUL": "#F596C8",
                 "KVY": "#2B4562",
                 "LAT": "#0082fa",
                 "LEC": "#DC0000",
                 "MAG": "#787878",
                 "NOR": "#FF8700",
                 "OCO": "#FFF500",
                 "PER": "#F596C8",
                 "RAI": "#9B0000",
                 "RIC": "#FFF500",
                 "RUS": "#0082fa",
                 "SAI": "#FF8700",
                 "STR": "#F596C8",
                 "VER": "#1E41FF",
                 "VET": "#DC0000",
            }
    )

    fig_min.update_traces(textposition='outside')

    fig_min.update_traces(opacity=0.85)

    fig_min.update_traces( marker_line_color='white',marker_line_width=1.5)

    fig_min.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

    fig_min.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

    fig_min.update_yaxes(range=[4.5, 0])

    fig_min.update_layout(
        xaxis={'side': 'top'},
        yaxis={'side': 'left'}
    )

    fig_min.update_xaxes(title_text='')      # Remove axis titles
    fig_min.update_yaxes(title_text='')

    fig_min.update_xaxes(
            title_standoff = 75
    )

    fig_min.update_layout(
        title_x=0.5,
        margin=dict(l=60, r=5, t=110, b=30),
        hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=15)),
        xaxis = dict(tickfont = dict(size=15)),
        font=dict(
            family="PT Sans Narrow",
            size=12,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    fig_min.update_yaxes(ticksuffix = "  ")

    fig_min.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)
    fig_min.write_html("{} {} Driver Fastest Lap Plot.html".format(year_multi,i))

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_9977/4204735011.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_9977/4204735011.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Index(['VER', 'HAM', 'BOT', 'LEC', 'ALB', 'VET', 'STR', 'HUL', 'KVY', 'NOR',
       'GAS', 'RIC', 'SAI', 'OCO', 'RUS', 'LAT', 'GRO', 'MAG', 'RAI', 'GIO'],
      dtype='object', name='Driver')
Index(['VER', 'BOT', 'HAM', 'ALB', 'NOR', 'RIC', 'SAI', 'LEC', 'GAS', 'OCO',
       'KVY', 'STR', 'VET', 'RAI', 'RUS', 'PER', 'GIO', 'LAT', 'MAG', 'FIT'],
      dtype='object', name='Driver')


/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_9977/4204735011.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_9977/4204735011.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Index(['BOT', 'HAM', 'NOR', 'PER', 'SAI', 'VER', 'LEC', 'ALB', 'GAS', 'OCO',
       'GIO', 'RAI', 'VET', 'KVY', 'STR', 'RIC', 'LAT', 'RUS', 'GRO', 'MAG'],
      dtype='object', name='Driver')
Index(['VER', 'HAM', 'PER', 'ALB', 'BOT', 'SAI', 'NOR', 'GAS', 'RIC', 'OCO',
       'LEC', 'KVY', 'VET', 'LAT', 'RUS', 'GIO', 'RAI', 'MAG'],
      dtype='object', name='Driver')


/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_9977/4204735011.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_9977/4204735011.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



...

Index(['BOT', 'VER', 'PER', 'HAM', 'RIC', 'LEC', 'KVY', 'ALB', 'OCO', 'GIO',
       'MAG', 'GAS', 'GRO', 'NOR', 'LAT', 'VET', 'RAI', 'RUS'],
      dtype='object', name='Driver')
Index(['OCO', 'RUS', 'BOT', 'RIC', 'SAI', 'KVY', 'GAS', 'PER', 'VET', 'GIO',
       'NOR', 'FIT', 'STR', 'AIT', 'ALB', 'MAG', 'RAI', 'LAT'],
      dtype='object', name='Driver')


/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_9977/4204735011.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_9977/4204735011.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Index(['HAM', 'BOT', 'RIC', 'LAT', 'VER', 'STR', 'RAI', 'KVY', 'SAI', 'OCO',
       'PER', 'GIO', 'ALB', 'RUS', 'GAS', 'NOR', 'VET', 'MAG', 'GRO'],
      dtype='object', name='Driver')
Index(['nan', 'SAI'], dtype='object', name='Driver')


/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_9977/4204735011.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_9977/4204735011.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Index(['SAI', 'HAM', 'GAS', 'LEC', 'NOR', 'VET', 'ALB', 'VER', 'RUS', 'STR',
       'PER', 'OCO', 'BOT', 'RIC', 'MAG', 'RAI', 'KVY', 'GRO', 'LAT'],
      dtype='object', name='Driver')
Index(['BOT', 'RAI', 'HAM', 'ALB', 'NOR', 'PER', 'KVY', 'LEC', 'RIC', 'RUS',
       'GRO', 'STR', 'VET'],
      dtype='object', name='Driver')


In [20]:
# #Get AVERAGE OF ALL LAP TIMES
# transformed_laps['avg_all'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Driver']).transform(np.mean)

# fastest_lap_avg = transformed_laps["avg_all"].min()
# transformed_laps['Delta_Avg_All']= transformed_laps["avg_all"] - (fastest_lap_avg)
# transformed_laps['Delta_Percent_Avg']= (transformed_laps["avg_all"]/(fastest_lap_avg)-1)*100
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg']+100
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg'].round(3)
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg'].astype(str)
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg']+"%"
# transformed_laps_relative_avg_all=transformed_laps[["Driver","Delta_Avg_All","Delta_Percent_Avg"]].drop_duplicates()
# transformed_laps_relative_avg_all=transformed_laps_relative_avg_all.sort_values("Delta_Avg_All")
# transformed_laps_relative_avg_all

In [201]:
# for index, row in transformed_laps_relative_avg_all.iterrows():
#   if transformed_laps_relative_avg_all.loc[index, 'Delta_Avg_All']==0.0:
#     transformed_laps_relative_avg_all.loc[index, 'Delta_Avg_All']=0.001
# transformed_laps_relative_avg_all

In [202]:
# fig=px.bar(
#     transformed_laps_relative_avg_all,
#     x="Driver",
#     y="Delta_Avg_All",
#     color='Driver',
#     template="xgridoff",
#     title="<b>Driver Pace (Average Lap Times) for the {} {}</b>".format(year,gp),
#     text="Delta_Percent_Avg",
#     height=700, width=1200,
#     color_discrete_map={
#                  "ALB": "#64C4FF",
#                  "ALO": "#229971",
#                  "BOT": "#52e252",
#                  "GAS": "#0093cc",
#                  "HAM": "#27F4D2",
#                  "HUL": "#B6BABD",
#                  "LEC": "#E80020",
#                  "MAG": "#B6BABD",
#                  "NOR": "#FF8000",
#                  "OCO": "#0093cc",
#                  "PER": "#3671C6",
#                  "PIA": "#FF8000",
#                  "RIC": "#6692FF",
#                  "RUS": "#27F4D2",
#                  "SAI": "#E80020",
#                  "SAR": "#37BEDD",
#                  "STR": "#229971",
#                  "TSU": "#6692FF",
#                  "VER": "#3671C6",
#                  "ZHO": "#52e252"
#           }
# )

# fig.update_traces(textposition='outside')

# fig.update_traces(opacity=0.85)

# fig.update_traces( marker_line_color='white',marker_line_width=1.5)

# fig.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

# fig.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

# fig.update_yaxes(range=[3.5, 0])

# fig.update_layout(
#     xaxis={'side': 'top'},
#     yaxis={'side': 'left'}
# )

# fig.update_xaxes(title_text='')      # Remove axis titles
# fig.update_yaxes(title_text='')

# fig.update_xaxes(
#         title_standoff = 75
# )

# fig.update_layout(
#     hoverlabel=dict(
#     bgcolor="white",
#     font_size=16,
#     font_family="PT Sans Narrow"
#     ),
#     title_x=0.5,
#     margin=dict(l=60, r=5, t=110, b=30),
#     yaxis = dict(tickfont = dict(size=15)),
#     xaxis = dict(tickfont = dict(size=15)),
#     font=dict(
#         family="PT Sans Narrow",
#         size=12,
#         color="Black"
#     ),
#     title_font_family="PT Sans Narrow"
# )
# fig.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)
# fig.show()

In [166]:
# fig.write_html("{} {} Driver Avg Lap Plot.html".format(year,gp))